# Equal-Weight S&P 500 Index Fund
### Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [2]:
# Import packages
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

### Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course.

There's a static version of the S&P 500 constituents available here. Click this link to download them now. Move this file into the starter-files folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


### Acquiring an API Token
Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

Click here to download your secrets.py file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [1]:
from secrets_token import IEX_CLOUD_API_TOKEN

### Making Our First API Call
Now it's time to structure our API calls to IEX cloud.

We need the following information from the API:

Market capitalization for each stock
Price of each stock

In [4]:
print(IEX_CLOUD_API_TOKEN)

pk_aebda4382ff24acd8520824a42050e25


In [5]:
symbol = 'AAPL'
api_url = f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}"

# Print the entire response, including headers and content
response = requests.get(api_url)
print("*** RESPONSE ***")
print(f"Status code: {response.status_code}")
print(f"Headers: {response.headers}")
print("Content:")
print(response.content)

# Try to decode the JSON (may still raise an error)
try:
    # If successful, access and print data from the JSON
    data = response.json()
    print("*** Decoded JSON Data ***")
    print(data)
except JSONDecodeError:
    print("Error decoding JSON")

# Additional tip: check the Content-Type header
print(f"\nContent-Type header: {response.headers['Content-Type']}")


*** RESPONSE ***
Status code: 200
Headers: {'Server': 'nginx', 'Date': 'Wed, 27 Dec 2023 03:33:34 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'ctoken=6c74b3ad7c5d43b7a79888bd9421506d; Max-Age=43200; Path=/; Expires=Wed, 27 Dec 2023 15:33:34 GMT', 'iexcloud-request-id': '4caca4e0ae56', 'iexcloud-messages-used': '1', 'iexcloud-credits-used': '1', 'iexcloud-premium-messages-used': '0', 'iexcloud-premium-credits-used': '0', 'X-Content-Type-Options': 'nosniff', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=15768000, max-age=15768000', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, OPTIONS', 'Access-Control-Allow-Headers': 'Origin, X-Requested-With, Content-Type, Accept, Request-Source', 'Cache-Control': 'no-cache'}
Content:
b'[{"avgTotalVolume":54011435,"calculationPrice":"close","change":-0.55,"changePercent":-0.00

### Parsing Our API Call
The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy.

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data[0]['latestPrice']
market_cap = data[0]['marketCap']

### Adding Our Stocks Data to a Pandas DataFrame
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy


In [8]:
final_dataframe = pd.concat([final_dataframe, pd.Series(
    [symbol, price, market_cap, 'N/A'], index=my_columns)], ignore_index=True)


### Looping Through The Tickers in Our List of Stocks
Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)  # Create empty DataFrame

for stock in stocks['Ticker']:
    api_url = f"https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()

    # Create a new row with correct column names
    new_row = pd.Series(
        [stock, data[0]['latestPrice'], data[0]['marketCap'], 'N/A'], index=my_columns
    )

    # Append the new row to the DataFrame
    final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)


In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,139.57,40894582516,N/A
1,AAL,14.31,9352165271,N/A
2,AAP,61.25,3644496888,N/A
3,AAPL,193.6,3011012787200,N/A
4,ABBV,154.94,273552368010,N/A
...,...,...,...,...
500,YUM,130,36440068470,N/A
501,ZBH,120.56,25194714518,N/A
502,ZBRA,269.41,13836790914,N/A
503,ZION,44,6518547156,N/A


### Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f"https://apis.iex.cloud/v1/data/core/QUOTE,FUNDAMENTALS,NEWS/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        # Find the dictionary with the matching symbol
        matching_dict = next((item for item in data if item["symbol"] == symbol), None)
#         for stock_data in data:
        try:
            symbol = matching_dict["symbol"]
            latest_price = matching_dict["latestPrice"]
            market_cap = matching_dict["marketCap"]
        except KeyError:
            latest_price = ""
            market_cap = ""

        # Create a new row with correct column names
        new_row = pd.Series(
                    [symbol, latest_price,market_cap, 'N/A'], index=my_columns
                )

        # Append the new row to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
            
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy
0,A,139.57,40894582516,N/A
1,AAL,14.31,9352165271,N/A
2,AAP,61.25,3644496888,N/A
3,AAPL,193.6,3011012787200,N/A
4,ABBV,154.94,273552368010,N/A
...,...,...,...,...
500,YUM,130,36440068470,N/A
501,ZBH,120.56,25194714518,N/A
502,ZBRA,269.41,13836790914,N/A
503,ZION,44,6518547156,N/A


### Calculating the Number of Shares to Buy
As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [20]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [21]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalisation,Number of Shares to Buy,Number Of Shares to Buy
0,A,139.57,40894582516,14,0.0
1,AAL,14.31,9352165271,138,0.0
2,AAP,61.25,3644496888,32,0.0
3,AAPL,193.6,3011012787200,10,0.0
4,ABBV,154.94,273552368010,12,0.0
...,...,...,...,...,...
500,YUM,130,36440068470,15,0.0
501,ZBH,120.56,25194714518,16,0.0
502,ZBRA,269.41,13836790914,7,0.0
503,ZION,44,6518547156,45,0.0


### Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

#### Initializing our XlsxWriter Object

In [22]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our .xlsx File
Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* $XX.XX format for stock prices
* $XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our .xlsx File
We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself".

Let's simplify this by putting it in 2 loops:

In [24]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

### Saving Our Excel Output
Saving our Excel file is very easy:

In [26]:
writer.close()